# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Review: First-order autonomous linear discrete-time systems

Recall the case of a pair of state variables that evolve discretely in time as a first-order autonomous linear system.

$$
  \begin{array}{rcl}
    x_t & = & a_0 x_{t-1} + b_0 y_{t-1} \\
    y_t & = & c_0 x_{t-1} + d_0 y_{t-1}
  \end{array}
  \implies
  \left(\begin{matrix} x_t \\ y_t \end{matrix}\right)
  = \underbrace{\left(\begin{matrix} a_0 & b_0 \\ c_0 & d_0 \end{matrix}\right)}_{\equiv \, A}
    \cdot \left(\begin{matrix} x_{t-1} \\ y_{t-1} \end{matrix}\right)
  = A \cdot \left(\begin{matrix} x_{t-1} \\ y_{t-1} \end{matrix}\right)
  = A^t \cdot \left(\begin{matrix} x_0 \\ y_0 \end{matrix}\right).
$$

Here is an implementation of a state transition function and subsequent simulation, for the case of $a_0 = d_0 = 0$ and $b_0 = -c_0 = 1.0$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def F (y, a0=0.0, b0=0.0, c0=0.0, d0=0.0):
    """State transition for a 1st-order autonomous linear system."""
    A = np.array ([[a0, b0], [c0, d0]])
    return A.dot (y)

def sim_F (t_max, y0, **args):
    """Simulates the system defined by `F(...)`."""
    Z = np.zeros ((2, t_max+1))
    Z[:, 0] = y0
    for t in range (t_max):
        Z[:, t+1] = F (Z[:, t], **args)
    return Z
        
def plot_sim_F (t_max, y0, **args):
    """Plots results of `sim_F()`."""
    Z = sim_F (t_max, y0, **args)
    z_min = np.min (Z)
    z_max = np.max (Z)
    
    fig = plt.figure (figsize=(10, 5))
    plt.subplot (1, 2, 1)
    plt.plot (Z[0, :], 'o-')
    plt.plot (Z[1, :], '*--')
    plt.axis ([-1, t_max+1, 1.1*z_min, 1.1*z_max])
    plt.title ('Time series')
    
    plt.subplot (1, 2, 2)
    plt.plot (Z[0, :], Z[1, :], '*-')
    plt.axis ('square')
    plt.axis (1.1*np.array ([z_min, z_max, z_min, z_max]))
    plt.title ('Phase space')
    
plot_sim_F (25, np.array ([1., -1.]), b0=1., c0=-1.)
np.linalg.eigvals (np.array ([[0., 1.], [-1., 0.]]))

# ODEs for Valentine's Day

When time is continuous, you will need a different way to model systems. The most common approach in the physical sciences and engineering is to turn to differential equations.

Let's start with _linear_ systems of ordinary differential equations, or ODEs, which have the form,

$$
  \dfrac{d}{dt} \vec{y}(t) = A \cdot \vec{y}(t),
$$

where $A$ is a matrix with (constant) coefficients. The goal of today's notebook is to help build your intuition through a lighthearted exercise, namely, modeling the "dynamics" of love! The specific example follows [an exercise](http://www.jstor.org/stable/2690328) suggested originally by Steven Strogatz.

## The setup

Romeo and Juliet are looking for love and find one another. Let,

* $R(t) \equiv$ Romeo's love (positive values) or hate (negative values) for Juliet; and
* $J(t) \equiv$ Juliet's love or hate for Romeo,

where both quantities are continuous functions of (continuous) time $t$.

Initially, Romeo immediately falls in love with Juliet, whereas Juliet has no opinion. Let's model this scenario as the initial condition of $R(0) = 1$ and $J(0) = 0$.

Romeo is the type of lover who mimics his partner: the more someone loves or hates him, the more he wants to return the same feeling. Juliet, by contrast, is fickle: the more someone loves her, the more she tends to dislike that person.

Let's model the dynamics of Romeo and Juliet's love by a system of ordinary differential equations---because that's everyone's first instinct (`#sarcasm`)---where the derivatives of $R(t)$ and $J(t)$ depend only linearly on $R(t)$ and $J(t)$:

$$
\begin{eqnarray}
  \begin{array}{rcl}
    \dfrac{dR(t)}{dt} & = & a_0 R(t) + b_0 J(t) \\
    \dfrac{dJ(t)}{dt} & = & c_0 R(t) + d_0 J(t)
  \end{array}
  & \qquad \implies \qquad &
  \begin{array}{rcl}
    \dfrac{d}{dt} \left(\begin{array}{c}
                    R(t) \\
                    J(t)
                  \end{array}\right)
    & = &
      \left(\begin{array}{cc}
        a_0 & b_0 \\
        c_0 & d_0
      \end{array}\right)
      \cdot
      \left(\begin{array}{c}
        R(t) \\
        J(t)
      \end{array}\right).
  \end{array}
\end{eqnarray}
$$

In general, the coefficients $a_0$, $b_0$, $c_0$, and $d_0$ could take on any values. But since Romeo is an eager beaver, it would be reasonable to let $a_0 = 0$ and $b_0 > 0$. For Juliet, we might choose her coefficients as $c_0 < 0$ and $d_0 = 0$.

Let's solve this system numerically. Let $\vec{y}(t) = \left(\begin{array}{c} y_0(t) \\ y_1(t) \end{array}\right) \equiv \left(\begin{array}{c} R(t) \\ J(t) \end{array}\right)$. Consider the system $\dfrac{d\vec{y}}{dt} = \vec{f}(t, \vec{y})$ for a suitable choice of $\vec{f}(\cdot, \cdot)$ corresponding to the "Romeo and Juliet" system.

In [ ]:
from scipy.integrate import odeint # ODE solver

**Exercise 1** (2 points). Write a function to compute $\vec{f}(t, \vec{y})$ for the Romeo and Juliet system. Then run the simulation code below. Do Romeo and Juliet find true love?

In [ ]:
def f_rj (y, t, a0, b0, c0, d0):
    """Implements the right-hand side of the R-J ODE system."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert (f_rj (np.array ([1.0, 0.0]), 0.2, 1., 2., 3., 4.) == np.array ([1., 3.])).all ()
assert (f_rj (np.array ([1.0, 0.0]), -1.7, 1., 2., 3., 4.) == np.array ([1., 3.])).all ()
assert (f_rj (np.array ([-1.0, 2.0]), -1.7, 1., 2., 3., 4.) == np.array ([3., 5.])).all ()

**Exercise 2** (3 points). Read about the basic SciPy ODE solver, [`odeint()`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.integrate.odeint.html), and "integrate" it into the simulation scaffold below. In particular, implement a function, `solve_rj_ode(y0, T, **args)`, which takes as input

- `y0[:n_y]`, the initial conditions (assuming `n_y` state variables);
- `T[:n_t]`, the time points at which to compute a solution (`n_t` time points);
- `**args`, arguments to pass to `f_rj`;

and returns

- `y[:n_y, :n_t]`, a `n_y` (number of variables) by `n_t` (number of time points) array containing the solution `y[:, k]` at each time point `T[k]`.

In [ ]:
def solve_rj_ode (y0, T, a0, b0, c0, d0):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test code (simulator)
def isim (r0=1.0, j0=-1.0, a0=0.0, b0=1.0, c0=-1.0, d0=0.0, t_max=20.0):
    """
    Simulates the R-J system up to a given maximum time
    and plots the results.
    """
    # Simulate:
    T = np.linspace (0, t_max, t_max*10+1)
    y0 = np.array ([r0, j0])
    Y = solve_rj_ode (y0, T, a0, b0, c0, d0)
    
    # Display results:
    r, j = Y[0, :], Y[1, :]
    plt.plot (T, r, 'r-', T, j, 'g-.')
    plt.title ('Strength of their affection')
    plt.xlabel ('Time')
    plt.legend (['Romeo, R(t)', 'Juliet, J(t)'], loc=0)
    
    y_equals_zero = np.zeros (T.shape)
    plt.plot (T, y_equals_zero, 'k-')
    
# Run a test case
isim (r0=-1.0, j0=1.0, a0=0.0, b0=1.0, c0=-1.0, d0=0.0, t_max=20)

**Exercise 3** (2 points). You should observe oscillatory behavior: Romeo and Juliet are engaged in an endless cycle of love and hate, where their feelings are mutual one-fourth of the time. Explain how you could have known to expect oscillatory behavior even before simulating the system.

YOUR ANSWER HERE

**Exercise 4** (2 points). Suppose our couple exhibits the following behavior.

1. Romeo is an _eager beaver_, meaning he gets excited _both_ by his partner's feelings of love _and_ by his own affectionate feelings. This case might be modeled as $a_0 > 0$ _and_ $b_0 > 0$.
2. Juliet is _cautiously self-aware_, meaning she resists Romeo's positive feelings but embraces her own positive feelings. This case might be modeled as $c_0 < 0$ and $d_0 > 0$.

Can an eager beaver and cautiously self-aware lover find romance? Analyze this scenario and then run a simulation. Use the following code cell for your analysis and simulation. Then, summarize your findings in the Markdown cell that follows the code cell.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**Exercise 5** (2 point). Come up with your own Romeo and Juliet scenario, by picking some initial conditions and values for the coefficients. Submit your choice, including a description of your scenario, here: http://j.mp/gtloveshack. Professor Vuduc will consider a prize (e.g., extra credit) for the most entertaining submissions!

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()